In [ ]:
import cobra.test
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import production_envelope, flux_variability_analysis, find_essential_reactions, find_essential_genes
import cobra.test
from cobra.flux_analysis import (single_gene_deletion, single_reaction_deletion, double_gene_deletion, double_reaction_deletion)
from cobra.flux_analysis.loopless import add_loopless, loopless_solution
import os
import pandas as pd
from Bio import SeqIO
import pickle
import numpy as np
import Bio
from Bio.KEGG import REST, Gene
import urllib.request
import re

<h3> finding additional essential genes in the mutant strain </h3>

In [42]:
model1 = cobra.io.read_sbml_model("final_model/WT_model_final.xml")
model2 = cobra.io.read_sbml_model("final_model/HP_model_final.xml")
essential1 = find_essential_genes(model1)
essential2 = find_essential_genes(model2)

rxns = ['2MBCOATA','3HAD100','3HAD110','3HAD120','3HAD121','3HAD130','3HAD140','3HAD141','3HAD150','3HAD160','3HAD161','3HAD50','3HAD60','3HAD70','3HAD80','3HAD90','3HADai110','3HADai130','3HADai150','3HADai170','3HADai70','3HADai90','3HADi100','3HADi110','3HADi120','3HADi130','3HADi140','3HADi150','3HADi160','3HADi170','3HADi60','3HADi70','3HADi80','3HADi90','3OAR100','3OAR110','3OAR120','3OAR121','3OAR130','3OAR140','3OAR141','3OAR150','3OAR160','3OAR161','3OAR50','3OAR60','3OAR70','3OAR80','3OAR90','3OARai110','3OARai130','3OARai150','3OARai170','3OARai70','3OARai90','3OARi100','3OARi110','3OARi120','3OARi130','3OARi140','3OARi150','3OARi160','3OARi170','3OARi60','3OARi70','3OARi80','3OARi90','3OAS100','3OAS110','3OAS120','3OAS121','3OAS130','3OAS140','3OAS141','3OAS150','3OAS160','3OAS161','3OAS50','3OAS60','3OAS70','3OAS80','3OAS90','3OASai110','3OASai130','3OASai150','3OASai170','3OASai70','3OASai90','3OASi100','3OASi110','3OASi120','3OASi130','3OASi140','3OASi150','3OASi160','3OASi170','3OASi60','3OASi70','3OASi80','3OASi90','5DOAN','5MTRt2','ACBIPGT','ACGK','ACHBS','ACLS','ACOTA','ACPPAT140','ACPPAT150','ACPPAT160','ACPPAT161','ACPPATai150','ACPPATai170','ACPPATi140','ACPPATi150','ACPPATi160','ACPPATi170','ADCL','ADCPS2','ADCS','ADCYRS','ADMDC','ADOCBLS','ADSK','ADSL1r','ADSL2r','ADSS','AGPAT140','AGPAT150','AGPAT160','AGPAT161','AGPATai150','AGPATai170','AGPATi140','AGPATi150','AGPATi160','AGPATi170','AGPR','AHCi','AHMMPS','AICART','AIRCr','AKP1','ALAALAr','ALAR','AMAOTr','ANPRT','AOXSr2','APG3PAT140','APG3PAT150','APG3PAT160','APG3PAT161','APG3PATai150','APG3PATai170','APG3PATi140','APG3PATi150','APG3PATi160','APG3PATi170','APRAUR','ARGSL','ARGSS','ASAD','ASNS1','ASPCT','ASPK','ASPTA','ATPPRT','ATPS4r','BIOMASS_SCO','BMOCOS','BMOGDS1','BMOGDS2','BPNT','BTS5','CA2t4','CBPS','CDPMEK','CHORM','CHORS','CIGAMS','CLPNS140','CLPNS150','CLPNS160','CLPNS161','CLPNSII140','CLPNSII150','CLPNSII160','CLPNSII161','CLPNSIIai150','CLPNSIIai170','CLPNSIIi140','CLPNSIIi150','CLPNSIIi160','CLPNSIIi170','CLPNSai150','CLPNSai170','CLPNSi140','CLPNSi150','CLPNSi160','CLPNSi170','CLt3_2','COBALT2t','COCHL','CPMPS','CPPPGO2','CU2t','CYRDAAT','CYRDAR','CYTK1','DAPDC','DAPE','DASYN140','DASYN150','DASYN160','DASYN161','DASYNai150','DASYNai170','DASYNi140','DASYNi150','DASYNi160','DASYNi170','DB4PS','DBTS','DDPA','DHAD1','DHAD2','DHCHCS','DHDPS','DHFR','DHFS','DHNPA2r','DHORTS','DHPPDA2','DHQS','DHQTi','DMATT','DMBZIDS','DM_4crsol_c','DM_5drib_c','DM_amob_c','DM_dalua_c','DPCOAK','DPR','DTMPK','DUTPDP','DXPRIi','DXPS','EGMEACPR','EPMEACPR','EX_5mtr_e','EX_ca2_e','EX_cl_e','EX_cobalt2_e','EX_cu2_e','EX_fe3_e','EX_glyc_e','EX_k_e','EX_mg2_e','EX_mn2_e','EX_mobd_e','EX_nh4_e','EX_ni2_e','EX_o2_e','EX_pi_e','EX_so4_e','EX_zn2_e','FCLT','FE3abc','FMNAT','FMNRx2','FRTT','G1PACT','G3PCT','G5SADs','GARFT','GCALDD','GF6PTA','GGTT','GLCS1','GLGC','GLNS','GLUPRT','GLYCt','GRTT','GTPCII2','HBADAS','HCO3E','HEMEAS','HEMEOS','HEPTT','HEXTT','HISTP','HMBS','HSK','HSST','HSTPT','IACGAMS2','IBCOATA','ICYSDS','IG3PS','IGAMD','IGPDH','IGPS','ILETA','IMPC','IPMD','IPPMIa','IPPMIb','IPPS','IVCOATA','LEUTA','LIPOS','LTHRK','LYSDC','MALACPMT','MCOATA','MCTP1A','MDDCP1','MECDPDH2','MECDPS','MEPCT','METAT','METS','MI1PP','MI1PS2','MI1PS','MMSAD3','MNXR113634','MNt2','MOADSUx','MOBDabc','MOCOS','MOGDS','MOHMT','MPTAT','MPTG','MPTS','MPTSS','MSHS','MTAN','MTHFC','MTHFD','NADK','NDPK1','NDPK2','NDPK3','NDPK4','NDPK5','NDPK7','NDPK8','NH4t','NNATr','NNDMBRT','NNDPR','O2t','OCT','OCTTT','OGMEACPD','OGMEACPR','OGMEACPS2','OMCDC','OMPDC','OPMEACPD','OPMEACPR','OPMEACPS','ORNDC','ORNTAC','ORPT','P5CR','PANTS','PAPA140','PAPA150','PAPA160','PAPA161','PAPAai150','PAPAai170','PAPAi140','PAPAi150','PAPAi160','PAPAi170','PAPPT3','PC11M','PC17M','PC20M','PC3BS','PC6AR','PC6AS','PC6YM','PC8XM','PCOATA','PGAMT','PGCD','PGI','PGMT','PHETA1','PINOS140','PINOS150','PINOS160','PINOS161','PINOSai150','PINOSai170','PINOSi140','PINOSi150','PINOSi160','PINOSi170','PMDPHT','PMEACPE','PMPK','PNTK','PPBNGS','PPCDC','PPND','PPNDH','PPPGO','PPTT','PRAGSr','PRAIS','PRAIi','PRAMPC','PRASCSi','PRATPP','PRFGS','PRMICI','PRPPS','PSCVT','PSD140','PSD150','PSD160','PSD161','PSDai150','PSDai170','PSDi140','PSDi150','PSDi160','PSDi170','PSERT','PSP_L','PSSA140','PSSA150','PSSA160','PSSA161','PSSAai150','PSSAai170','PSSAi140','PSSAi150','PSSAi160','PSSAi170','PTPATi','PYDXPS','QULNS','RBFK','RBFSa','RBFSb','RZ5PP','Rapamycin','S2FE2SR','S2FE2SS2','S2FE2SS','S2FE2ST','S4FE4SR','S4FE4ST','SCYSDS','SDPDS','SDPTA','SHCHD2','SHCHF','SHK3Dr','SHKK','SPMS','SPRMS','T2DECAI','TAGO','TAGS140','TAGS150','TAGS160','TAGS161','TAGSai150','TAGSai170','TAGSi140','TAGSi150','TAGSi160','TAGSi170','TECA1S','TECA2S','TECA3S','THDPS','THRPDC','THRS','THZPSN3','TMDSf','TMPK','TMPPP','TRE6PP','TRE6PS','TYRL','TYRTA','UAAGDS','UAG2E','UAGCVT','UAGDP','UAGPT3','UAMAS','UAPGR','UDCPDP','UDCPDPS','UDPG4E','UGMDDS','UMPK','UPP3S','UPPDC1','VALTA','ZNabc']

for rxn in rxns:
    a = str(model1.reactions.get_by_id(rxn).gene_reaction_rule)
    a_list = [model1.genes.get_by_id(gene) in essential1 for gene in a.split() if gene.startswith('NJM')]
    b = str(model2.reactions.get_by_id(rxn).gene_reaction_rule)
    b_list = [model2.genes.get_by_id(gene) in essential2 for gene in b.split() if gene.startswith('JDD')]
    
    if a.count('NJM') != b.count('JDD') and (True in a_list or True in b_list):
        print(rxn)
        print(model1.reactions.get_by_id(rxn).gene_reaction_rule)
        print(a_list)
        print(model2.reactions.get_by_id(rxn).gene_reaction_rule)
        print(b_list)
        print('-----------------------------------')

ARGSS
NJMEEBPL_09086
[True]
SCO7036
[]
-----------------------------------
ASNS1
NJMEEBPL_00505 or NJMEEBPL_00879 or NJMEEBPL_08219
[False, False, False]
JDDNEILB_00640
[True]
-----------------------------------
DHQTi
NJMEEBPL_01239 or NJMEEBPL_10171
[False, False]
JDDNEILB_07629
[True]
-----------------------------------
LYSDC
NJMEEBPL_09618
[True]
SCO7311
[]
-----------------------------------
MI1PP
NJMEEBPL_00429 or NJMEEBPL_02947
[False, False]
JDDNEILB_05915
[True]
-----------------------------------
MOBDabc
( NJMEEBPL_00385 or NJMEEBPL_00651 or NJMEEBPL_00716 or NJMEEBPL_01441 or NJMEEBPL_02481 or NJMEEBPL_03116 or NJMEEBPL_03133 or NJMEEBPL_04936 or NJMEEBPL_05507 or NJMEEBPL_05552 or NJMEEBPL_05743 or NJMEEBPL_07788 or NJMEEBPL_08779 or NJMEEBPL_09557 ) and NJMEEBPL_04937 and NJMEEBPL_04938
[False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, True]
( JDDNEILB_00084 or JDDNEILB_01071 or JDDNEILB_03116 or JDDNEILB_03308 or JDDNE